In [37]:
import pandas as pd
import numpy as np
import math as m
import random as r
from IPython.display import display, HTML

def isNegative(num):
    return  num < 0


def biseccion(threshold, fp, epsilon, numIterations, method):    
    finish = False
    fpp = fp.diff(x)
    while(not finish):
        values = pd.DataFrame(columns=('x0', 'x1', 'xp', 'f(x0)', 'f(x1)', 'f(xp)'))
        i = 0
        x0 = r.uniform(threshold[0],threshold[1])
        x1 = r.uniform(threshold[0],threshold[1])
        while(isNegative(fp.evalf(subs={x:x0})) == isNegative(fp.evalf(subs={x:x1}))):
            x0 = r.uniform(threshold[0],threshold[1])
            x1 = r.uniform(threshold[0],threshold[1])
        if(x1 < x0):
            aux = x1
            x1 = x0
            x0 = aux
        error = m.fabs(x0 - x1)
        while((i < numIterations) and (error > epsilon)):
            xp = x0 + (m.fabs(x0-x1)/2)
            f_x0 = fp.evalf(subs={x:x0})
            f_x1 = fp.evalf(subs={x:x1})
            biseccion = fp.evalf(subs={x:xp})
            values.loc[len(values)] = [x0, x1, xp, f_x0, f_x1, biseccion]
            if(biseccion == 0):
                return values
            elif(isNegative(f_x0) == isNegative(biseccion)):
                x0 = xp
            elif(isNegative(f_x1) == isNegative(biseccion)): 
                x1 = xp
            i = i + 1 
            error = m.fabs(x0 - x1)
        if((method == 0 and fpp.evalf(subs={x:xp})>0) or (method == 1 and fpp.evalf(subs={x:xp})<0)):
            finish = True
    return values 

def newtonRaphson(threshold, fp, epsilon, numIterations, method):
    fpp = fp.diff(x)
    finish = False
    while(not finish):
        values = pd.DataFrame(columns=('x', 'new_x', 'f(x)', 'f(x_new_x)'))
        error = -1
        i = 0
        point = r.uniform(threshold[0],threshold[1])
        while((i < numIterations) and (error > epsilon or error == -1)):
            f_point = fp.evalf(subs={x:point})
            fp_point = fpp.evalf(subs={x:point})
            point2 = point - (f_point)/(fp_point)
            f_newPoint = fp.evalf(subs={x:point2})
            error = m.fabs(f_newPoint)
            values.loc[len(values)] = [point, point2, f_point, f_newPoint]
            point = point2
            i = i + 1
        if((method == 0 and fpp.evalf(subs={x:point})>0) or (method == 1 and fpp.evalf(subs={x:point})<0)):
            finish = True
    return values

def getValue(values, function):
    optimalValue = values.loc[len(values) -1]
    if(function == "biseccion"):
        return optimalValue.loc['xp']
    elif(function == "newtonRaphson"):
        return optimalValue.loc['new_x']
    raise Exception('metodo ' + function + ' no existe!')
    
    
def optimization(threshold, f, epsilon, numIterations, objective, function):
    fp = f.diff(x)
    if(function == "newtonRaphson"):
        values = newtonRaphson(threshold, fp, epsilon, numIterations, objective)
    elif(function == "biseccion"):
        values = biseccion(threshold, fp, epsilon, numIterations, objective)
    optimalVaule = getValue(values, function)
    
    return HTML(""""<h1 style="color:red;" align="center">""" + function + """</h1>
        <h2>Historial</h2>
        """ + values.to_html() + """
        <h2>Valor optimo</h2>
        <p>""" + str(optimalVaule))

In [40]:
from sympy.abc import x

y = -(0.3 * (x ** 4)) + (1.2 * (x ** 3)) - (1.8 * (x ** 2)) + (4 * x) 

v = optimization((-2,4), y, 0.0001, 100, 1, "newtonRaphson")
v


,x,new_x,f(x),f(x_new_x)
0,0.477561,3.50130684490690,2.97111526537038,-15.9794193836761
1,3.50130684490690,2.79185233932245,-15.9794193836761,-4.10379520755110
2,2.79185233932245,2.43681114716253,-4.10379520755110,-0.759428817178346
3,2.43681114716253,2.33462660707263,-0.759428817178346,-0.0527294272450617
4,2.33462660707263,2.32640359371276,-0.0527294272450617,-0.000324213740547933
5,2.32640359371276,2.32635240460777,-0.000324213740547933,-1.25120272415462e-8
